In [7]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
from misc_functions import calculate_RTs


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np
import copy

from misc_functions import calculate_RTs, save_sig_chans, save_channels_to_file
import pickle

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

### actually make the baselines so we can compare them later

In [2]:
def get_baseline(inst: mne.io.BaseRaw, base_times: tuple[float, float]):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")
    del inst
    return HG_base


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5), base_times=(-0.5, 0), LAB_root=None, channels=None, full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.
    
    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task. 
    High gamma (HG) will be computed, and statistics will be calculated and plotted. 
    The results will be saved to output files.
    """
    pass

sub='D0057'
task='GlobalLocal'
output_name="response_experimentStartBase1secTo101Sec"
event="Response"
times=(-1, 1.5)
base_times=(1, 101)
LAB_root=None
channels=None
full_trial_base=False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

try:
    good = crop_empty_data(filt)
except ValueError as ve:
    print(f"Error encountered during cropping: {ve}")

print(f"good channels before dropping bads: {len(good.ch_names)}")
print(f"filt channels before dropping bads: {len(filt.ch_names)}")

good.info['bads'] = channel_outlier_marker(good, 3, 2)
print("Bad channels in 'good':", good.info['bads'])

filt.drop_channels(good.info['bads']) #this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
good.drop_channels(good.info['bads'])

print("Bad channels in 'good' after dropping once:", good.info['bads'])

print(f"good channels after dropping bads: {len(good.ch_names)}")
print(f"filt channels after dropping bads: {len(filt.ch_names)}")


HG_base = get_baseline(filt, base_times)
good.load_data()

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


good channels before dropping bads: 178
filt channels before dropping bads: 178
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Bad channels in 'good': ['RAMT8', 'RPI16', 'LAMT12']
Bad channels in 'good' after dropping once: []
good channels after dropping bads: 175
filt channels after dropping bads: 175
Reading 0 ... 8243199  =      0.000 ...  4025.000 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus

100%|██████████| 1/1 [00:53<00:00, 53.27s/it]


Reading 0 ... 5983997  =      0.000 ...  2921.874 secs...


<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 175 x 5983998 (2921.9 s), ~7.80 GB, data loaded>

In [5]:
# Create a baseline EpochsTFR using the stimulus event
base_times=(-0.5,0)
adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base_stimulus = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base_stimulus, "0.5s")

Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75', 'experimentStart']
Not setting metadata


448 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 448 events and 3073 original time points ...
0 bad epochs dropped


100%|██████████| 448/448 [04:24<00:00,  1.70it/s]


Number of events,448
Events,Stimulus/c25/n25: 1Stimulus/c25/r25: 63Stimulus/c25/r75: 23Stimulus/c25/s25: 20Stimulus/c25/s75: 61Stimulus/c75/r25: 24Stimulus/c75/r75: 11Stimulus/c75/s25: 4Stimulus/c75/s75: 17Stimulus/i25/n75: 1Stimulus/i25/r25: 21Stimulus/i25/r75: 5Stimulus/i25/s25: 7Stimulus/i25/s75: 22Stimulus/i75/n25: 1Stimulus/i75/n75: 1Stimulus/i75/r25: 60Stimulus/i75/r75: 17Stimulus/i75/s25: 23Stimulus/i75/s75: 66
Time range,-0.500 – 0.000 s
Baseline,off


In [10]:
data = HG_base.get_data()

mean = np.mean(data, axis=(0,2))
stdev = np.std(data,axis=(0,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base.ch_names

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(channel_names, mean, yerr=stdev, align='center', alpha=0.75, ecolor='black', capsize=5)

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.title('Mean and Standard Deviation of Channels')
plt.xticks(rotation=90)  # Rotate the channel names for better readability

# Display the plot
plt.tight_layout()
plt.show()


Mean:  [3.84630322e-06 3.29684330e-06 3.31282825e-06 3.60361747e-06
 4.42215705e-06 7.41808568e-06 2.84947609e-06 2.18214122e-06
 2.27253409e-06 2.55375306e-06 3.06813133e-06 5.57559093e-06
 1.03559997e-05 9.65731638e-06 8.66877990e-06 6.64132752e-06
 4.57139391e-06 3.61304781e-06 3.44952659e-06 4.78302741e-06
 3.50475003e-06 3.34690632e-06 2.33347931e-06 2.54583200e-06
 3.09513135e-06 3.95237442e-06 6.42933658e-06 1.36428357e-05
 1.30121400e-05 1.41465485e-05 6.30349080e-06 8.51701921e-06
 8.15496148e-06 7.65769407e-06 8.45221358e-06 8.17010856e-06
 7.82943500e-06 8.62322971e-06 9.44030009e-06 7.32383139e-06
            nan 9.99880586e-06 5.18744102e-06 5.04847399e-06
 6.14913545e-06 7.16740083e-06 3.70821071e-06 4.17609935e-06
 3.05756317e-06 2.28939142e-06 2.26979705e-06 2.49318017e-06
 2.95730527e-06 4.06850892e-06 9.77091522e-06 1.15611172e-05
 9.39998255e-06 6.72327816e-06 5.32837368e-06 6.14017824e-06
 4.70478137e-06 2.95435123e-06 2.58999636e-06 2.56373340e-06
 2.79294049e-06 4

KeyboardInterrupt: 

### average across trials, channels, and time points

In [14]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

Mean:  5.535233531799012e-06
Standard Deviation:  3.4144343686885302e-06


In [15]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

Mean:  5.85340818925691e-06
Standard Deviation:  2.9913158867295083e-06
